In [2]:
!pip install transformers
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.7 MB/s eta 0:00:00


In [3]:
# Imports

import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline

### Data Cleaning

***Trump & Republican Debate***

In [4]:
from google.colab import drive

# mount our drive, can access it like a local file path
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
### Load CSV - GOP
rep_df = pd.read_csv("/content/drive/MyDrive/project-4/rep_df_cleaned.csv")

In [6]:
rep_df.head()

,Unnamed: 0,Comment,timestamp,sentiment,sentiment_label,year_month
0,464,I totally support and endorse Donald.,2016-06-25 10:54:46+00:00,0.6764,Positive,2016-06
1,465,Is that Tito Ortiz from the UFC next to the la...,2016-06-07 21:11:38+00:00,0.5984,Positive,2016-06
2,466,"I actually can't watch this, first 20 minutes ...",2016-05-31 14:22:36+00:00,0.7506,Positive,2016-05
3,467,he is a coward,2016-05-31 08:16:06+00:00,-0.4588,Negative,2016-05
4,468,😴😴😴,2016-05-31 05:53:00+00:00,0.0000,Neutral,2016-05


In [7]:
rep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1614 non-null   int64  
 1   Comment          1614 non-null   object 
 2   timestamp        1614 non-null   object 
 3   sentiment        1614 non-null   float64
 4   sentiment_label  1614 non-null   object 
 5   year_month       1614 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 75.8+ KB


In [8]:
rep_df['timestamp'] = pd.to_datetime(rep_df['timestamp'])

In [9]:
rep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   Unnamed: 0       1614 non-null   int64              
 1   Comment          1614 non-null   object             
 2   timestamp        1614 non-null   datetime64[ns, UTC]
 3   sentiment        1614 non-null   float64            
 4   sentiment_label  1614 non-null   object             
 5   year_month       1614 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(3)
memory usage: 75.8+ KB


In [10]:
# Instantiating Hugging Face sentiment model
classifier = pipeline(
    "sentiment-analysis",
    model="siebert/sentiment-roberta-large-english",
    tokenizer="siebert/sentiment-roberta-large-english",
    truncation=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


In [14]:
# Applying Huggingface sentiment classification

rep_df["HF_Sentiment"] = rep_df["Comment"].apply(lambda x: classifier(x)[0]["label"])
rep_df["HF_Score"] = rep_df["Comment"].apply(lambda x: classifier(x)[0]["score"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [16]:
 rep_df['HF_Score_Confidence'] = rep_df['HF_Score']

In [18]:
rep_df = rep_df.drop(columns=['HF_Score'])

In [19]:
rep_df.head()

,Unnamed: 0,Comment,timestamp,sentiment,sentiment_label,year_month,HF_Sentiment,HF_Score_Confidence
0,464,I totally support and endorse Donald.,2016-06-25 10:54:46+00:00,0.6764,Positive,2016-06,POSITIVE,0.998792
1,465,Is that Tito Ortiz from the UFC next to the la...,2016-06-07 21:11:38+00:00,0.5984,Positive,2016-06,POSITIVE,0.988146
2,466,"I actually can't watch this, first 20 minutes ...",2016-05-31 14:22:36+00:00,0.7506,Positive,2016-05,NEGATIVE,0.999508
3,467,he is a coward,2016-05-31 08:16:06+00:00,-0.4588,Negative,2016-05,NEGATIVE,0.999319
4,468,😴😴😴,2016-05-31 05:53:00+00:00,0.0000,Neutral,2016-05,POSITIVE,0.997923


In [20]:
rep_df = rep_df.drop(columns=['Unnamed: 0'])

rep_df.to_csv('/content/drive/MyDrive/project-4/rep_df_HF.csv')

In [ ]:
!pip install matplotlib

In [11]:
dem_df = pd.read_csv("/content/drive/MyDrive/project-4/dem_df_cleaned.csv")

In [12]:
dem_df.head()

,Unnamed: 0,Comment,timestamp,sentiment,sentiment_label,year_month
0,21,www.infowars.com,2016-10-25 21:47:05+00:00,0.0000,Neutral,2016-10
1,22,She's so full of shit.,2016-10-20 03:25:18+00:00,-0.5945,Negative,2016-10
2,23,Empty promises and emotional rhetoric to persu...,2016-10-19 11:48:47+00:00,0.9325,Positive,2016-10
3,24,"Black live matter Hillary say so, then what ab...",2016-10-19 03:20:58+00:00,0.2845,Positive,2016-10
4,25,She cheated Bernie out of nomination.,2016-10-19 01:25:36+00:00,-0.5106,Negative,2016-10


In [13]:
dem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30269 entries, 0 to 30268
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       30269 non-null  int64  
 1   Comment          30269 non-null  object 
 2   timestamp        30269 non-null  object 
 3   sentiment        30269 non-null  float64
 4   sentiment_label  30269 non-null  object 
 5   year_month       30269 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.4+ MB


In [14]:
dem_df = dem_df.drop(columns=['Unnamed: 0'])

In [15]:
dem_df['timestamp'] = pd.to_datetime(dem_df['timestamp'])

In [16]:
dem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30269 entries, 0 to 30268
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   Comment          30269 non-null  object             
 1   timestamp        30269 non-null  datetime64[ns, UTC]
 2   sentiment        30269 non-null  float64            
 3   sentiment_label  30269 non-null  object             
 4   year_month       30269 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(3)
memory usage: 1.2+ MB


In [17]:
# Applying Huggingface sentiment classification

dem_df["HF_Sentiment"] = dem_df["Comment"].apply(lambda x: classifier(x)[0]["label"])
dem_df["HF_Label_Confidence"] = dem_df["Comment"].apply(lambda x: classifier(x)[0]["score"])

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [18]:
dem_df.head()

,Comment,timestamp,sentiment,sentiment_label,year_month,HF_Sentiment,HF_Label_Confidence
0,www.infowars.com,2016-10-25 21:47:05+00:00,0.0000,Neutral,2016-10,NEGATIVE,0.988892
1,She's so full of shit.,2016-10-20 03:25:18+00:00,-0.5945,Negative,2016-10,NEGATIVE,0.999483
2,Empty promises and emotional rhetoric to persu...,2016-10-19 11:48:47+00:00,0.9325,Positive,2016-10,NEGATIVE,0.997069
3,"Black live matter Hillary say so, then what ab...",2016-10-19 03:20:58+00:00,0.2845,Positive,2016-10,NEGATIVE,0.996861
4,She cheated Bernie out of nomination.,2016-10-19 01:25:36+00:00,-0.5106,Negative,2016-10,NEGATIVE,0.999420


In [19]:
dem_df.to_csv('/content/drive/MyDrive/project-4/dem_df_HF.csv')